- https://python.langchain.com/docs/modules/agents/quick_start

In [1]:
import re
from math import pi
from typing import Dict, Optional, Tuple, Type, Union

from langchain import hub
from langchain.agents import create_react_agent
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.memory import ConversationBufferMemory
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, tool
from langchain_community.llms import Ollama

# from langchain_openai import AzureChatOpenAI, OpenAI

In [2]:
# llm = AzureChatOpenAI(
#     azure_endpoint=os.environ["AZURE_ENDPOINT"],
#     openai_api_key=os.environ["OPENAI_API_KEY"],
#     openai_api_type="azure",
#     openai_api_version="2023-05-15",
#     deployment_name="gpt-4-32k",
#     temperature=0.1,
# )

In [24]:
llm = Ollama(model="mistral:latest", temperature=0.0)

## Get a prompt.

In [25]:
prompt = hub.pull("hwchase17/react")

In [26]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### Define a dummy locator.

In [27]:
class Locator:
    def __init__(self, location: Dict) -> None:
        """Initialize with location table"""
        self.locations = locations

    def locate(self, query: str) -> Tuple[float, float]:
        return self.locations.get(query.lower(), (None, None))

### Define the geocode tool input.

In [28]:
class GeocodeInput(BaseModel):
    query: str = Field(description="should be the location to search and geocode")

### Define the geocoder tool.

In [29]:
class GeocodeTool(BaseTool):
    name = "geocode_tool"
    description = """
    Use this tool to get the latitude and longitude of an address, a place or a country.
    """
    args_schema: Type[BaseModel] = GeocodeInput
    return_direct: bool = True
    locator: Locator = Field(exclude=True)

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> Tuple[float, float]:
        return self.locator.locate(query)

    async def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> Tuple[float, float]:
        raise NotImplementedError(f"{self.name} does not support async")

### Create a geocoder tool instance.

In [30]:
locations = {
    "beirut, lebanon": (33.8938, 35.5018),
    "beirut": (33.8938, 35.5018),
    "boston": (42.3601, -71.0589),
    "singapore": (1.3521, 103.8198),
}

geocode_tool = GeocodeTool(locator=Locator(locations))

In [31]:
# geocode_tool.run("beirut")

### Create an extent calculator, input and tool.

In [32]:
class ExtentCalculator:
    def calculate(
        self,
        lon: float,
        lat: float,
        dist: float,
    ) -> Tuple[float, float, float, float]:
        return lon - dist, lat - dist, lon + dist, lat + dist

### Define Extent Tool Inputs.

In [33]:
class ExtentInput(BaseModel):
    lon: float = Field(
        description="A longitude value",
    )
    lat: float = Field(
        description="A latitude value",
    )
    dist: float = Field(
        description="The extent width, height or distance",
    )

### Define Extent Tool.

In [34]:
class ExtentTool(BaseTool):
    name = "extent_tool"
    description = """
    Use this tool to calculate an extent or a rectangle whose dimensions are proportional to a given distance
    and is centered around given latitude and longitude values.
    """
    args_schema: Type[BaseModel] = ExtentInput
    return_direct: bool = True
    calculator: ExtentCalculator = Field(exclude=True)

    def _run(
        self,
        lon: float,
        lat: float,
        dist: float,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        return self.calculator.calculate(lon, lat, dist)

    async def _arun(
        self,
        lon: float,
        lat: float,
        dist: float,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        raise NotImplementedError(f"{self.name} does not support async")

### Create an extent tool instance.

In [35]:
extent_tool = ExtentTool(calculator=ExtentCalculator())

In [36]:
def extract_float_from_string(input_string):
    # Regular expression pattern to match a number (integer or decimal)
    pattern = r"\d+(\.\d+)?"

    # Searching for the pattern in the input string
    match = re.search(pattern, input_string)

    # If a match is found, convert it to float and return
    return float(match.group()) if match else None


# Example usage
# extract_float_from_string("7.81 mm")

In [37]:
@tool
def calculate_circumference(radius: Union[str, int, float]) -> float:
    """Use this tool when you need to calculate a circumference  of a circle using the radius."""

    # Regular expression pattern to match a number (integer or decimal)
    pattern = r"\d+(\.\d+)?"

    # Searching for the pattern in the input string
    match = re.search(pattern, radius)
    return float(match.group()) * 2.0 * pi if match else float("NaN")

In [38]:
@tool
def calculate_area(radius: Union[str, int, float]) -> float:
    """Use this tool when you need to calculate an area of a circle using the radius."""

    # Regular expression pattern to match a number (integer or decimal)
    pattern = r"\d+(\.\d+)?"

    # Searching for the pattern in the input string
    match = re.search(pattern, radius)
    rad = float(match.group())
    return rad * rad * pi if match else float("NaN")

### Define the tools.

In [39]:
tools1 = [calculate_circumference, calculate_area, geocode_tool, extent_tool]
# tools2 = load_tools(["llm-math"], llm=llm)

In [40]:
tools = tools1  # + tools2

In [41]:
agent = create_react_agent(llm, tools, prompt)

In [42]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=10,
    early_stopping_method="generate",
    max_tokens=256,
)

In [43]:
resp = agent_executor.invoke(
    {"input": "Calculate an extent of 2 degress around Beirut"}
)
resp["output"]



> Entering new AgentExecutor chain...
 To calculate an extent of 2 degrees around Beirut, I first need to find the latitude and longitude of Beirut using the geocode_tool. Then, I will use the extent_tool to calculate the extent based on the given degree value and the latitude and longitude of Beirut.

Action: geocode_tool
Action Input: Beirut(33.8938, 35.5018)


> Finished chain.


(33.8938, 35.5018)

In [ ]:
# resp = agent_executor.invoke(
#     {
#         "input": "What is the square root of the area of a circle with a 100 meter radius?"
#     }
# )
# resp["output"]